In [3]:
#sample for read-in to prep for normalization

import pandas as pd, numpy as np

#SrcDF contains all the items in the entire dataset
SrcDF = pd.read_csv("C:\\Users\\pconn\\OneDrive\\Desktop\\AFAD-Full\\afad_test.csv")
#these are the files from the overall dataset that were fed into the model
AFace = pd.read_csv("C:\\Users\\pconn\\OneDrive\\Desktop\\AFAD-Full\\afad_loc_clean.csv")
#This is the output from the deepface model
DFace = pd.read_csv("C:\\Users\\pconn\\OneDrive\\Desktop\\AFAD-Full\\deepfaceout.csv")
#This is the output from the fairface model
FFace = pd.read_csv("C:\\Users\\pconn\\OneDrive\\Desktop\\AFAD-Full\\test_outputs_afad.csv")
DFace.head(5)


,Unnamed: 0.6,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,filename,race,age,gender
0,0,0.0,0.0,0.0,NaN,NaN,NaN,./part1/100_1_0_20170110183726390.jpg,middle eastern,48,Man
1,1,1.0,1.0,0.0,NaN,NaN,NaN,./part1/100_1_2_20170105174847679.jpg,white,27,Man
2,2,2.0,2.0,0.0,NaN,NaN,NaN,./part1/100_1_2_20170110182836729.jpg,white,29,Man
3,3,3.0,3.0,0.0,NaN,NaN,NaN,./part1/101_1_2_20170105174739309.jpg,indian,43,Man
4,4,4.0,4.0,0.0,NaN,NaN,NaN,./part1/10_0_0_20161220222308131.jpg,asian,31,Man


This code is focused on the AFAD dataset and exploring what data I had readily available (only 500 out of may thousands in the overall dataset.

Below looks at the fairface dataset.  There's a column that contains the filename, so we need to remove any path information and clean that up so that we have the right info to do joins between our data.

For any columns associated with Fair Face, I propose that we add the FF_pred for "Fair Face Prediction" followed by whatever variable it's predicting.



In [4]:
#prepare fairface for merger
FFace['file'] = FFace['face_name_align'].str.split('/').str[-1]
FFace['file'] = FFace['file'].str.split('_').str[0]
FFace['file'] = FFace['file'].astype(str) + '.jpg'
FFace = FFace.drop(labels=['face_name_align','race_scores_fair_4','gender_scores_fair','race_scores_fair','age_scores_fair'],axis=1)
FFace.columns = ['FF_pred_race','FF_pred_race4','FF_pred_gender','FF_pred_age','file']
FFace.head(5)

,FF_pred_race,FF_pred_race4,FF_pred_gender,FF_pred_age,file
0,East Asian,Asian,Male,20-29,100055-0.jpg
1,East Asian,Asian,Female,20-29,101516-0.jpg
2,East Asian,Asian,Female,10-19,104446-0.jpg
3,East Asian,Asian,Female,20-29,104512-0.jpg
4,East Asian,Asian,Female,10-19,104640-0.jpg


Here, AFace is just a listing of the individual files that were available for evaluation.  This is useful to us pre-execution because it can:
    - Tell us how many images could be processed by one of the models.
    - Allows us to run the model on a subset of the data vs. all of it (i.e. I ran FairFace and DeepFace against approx 10K faces of UTK)
    - 

In [ ]:
#prepare the fed file listing for integrating everything together
AFace.columns=['file']
AFace['file'] = AFace['file'].str.split('/').str[-1]
AFace['present'] = True
AFace


,file,present
0,244515-0.jpg,True
1,109440-0.jpg,True
2,263909-3.jpg,True
3,270951-0.jpg,True
4,289788-1.jpg,True
...,...,...
486,258242-0.jpg,True
487,271805-0.jpg,True
488,278033-0.jpg,True
489,50737-1.jpg,True


In [ ]:
DFace = DFace.drop(labels=[DFace.columns[0]],axis=1)
DFace.columns = ['file','DF_pred_race','DF_pred_age','DF_pred_gender']
DFace['file'] = DFace['file']=DFace['file'].str.split('/').str[-1]
DFace['DF_pred_race'] = DFace['DF_pred_race'].str.title()
DFace['DF_pred_gender'] = np.where(DFace['DF_pred_gender']=='Man','Male','Female')
#print(np.max(DFace['DF_pred']))

In [ ]:
SrcDF.columns = ['file','path','recorded_age','recorded_gender']
SrcDF.head(5)
#print(np.max(SrcDF['recorded_age']),np.min(SrcDF['recorded_age']),np.average(SrcDF['recorded_age']))

,file,path,recorded_age,recorded_gender
0,406208-0.jpg,39/112/406208-0.jpg,24,female
1,470804-0.jpg,39/112/470804-0.jpg,24,female
2,411657-2.jpg,39/112/411657-2.jpg,24,female
3,471528-2.jpg,39/112/471528-2.jpg,24,female
4,398608-1.jpg,39/112/398608-1.jpg,24,female


In [ ]:
init_merge = SrcDF.merge(AFace,on='file',how='left').dropna()
#init_merge = init_merge.merge(right=FFace,on='file',how='left')
#init_merge = init_merge.merge(right=DFace,on='file',how='left')
init_merge

,file,path,recorded_age,recorded_gender,present
14378,244515-0.jpg,19/112/244515-0.jpg,4,female,True
14379,109440-0.jpg,19/112/109440-0.jpg,4,female,True
14380,263909-3.jpg,19/112/263909-3.jpg,4,female,True
14386,270951-0.jpg,19/112/270951-0.jpg,4,female,True
14388,289788-1.jpg,19/112/289788-1.jpg,4,female,True
...,...,...,...,...,...
15558,258242-0.jpg,19/112/258242-0.jpg,4,female,True
15559,271805-0.jpg,19/112/271805-0.jpg,4,female,True
15564,278033-0.jpg,19/112/278033-0.jpg,4,female,True
15565,50737-1.jpg,19/112/50737-1.jpg,4,female,True


In [ ]:
np.max(init_merge['recorded_age'])

25